# Phi 3 mini test (microsoft llm)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm

# Load model and tokenizer
model_name = "microsoft/phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

# Function to split text into chunks
def chunk_text(text, max_words=500):
    words = text.split()
    chunks = []
    for i in range(0, len(words), max_words):
        chunks.append(" ".join(words[i:i+max_words]))
    return chunks

# Function to create a strict no-paraphrasing prompt
def create_no_rephrase_prompt(text_chunk):
    return f"""
You are a meticulous text proofreader. 
Correct ONLY spelling, punctuation, and grammatical errors. 
Do NOT change wording, sentence structure, or style unless absolutely necessary. 
Preserve all original text, tone, and sentence order.
Even if a sentence is incomplete, it is importance that we keep the original document intact.
Output only the text without any additional statement

Text:
\"\"\"{text_chunk}\"\"\"

Corrected Text:
"""

# Function to correct a single chunk
def correct_chunk(chunk):
    prompt = create_no_rephrase_prompt(chunk)
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    output_ids = model.generate(
        **inputs,
        max_new_tokens=1000,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )
    corrected_text = tokenizer.decode(output_ids[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    return corrected_text.strip()

# Main function to correct full transcript
def correct_transcript(text):
    chunks = chunk_text(text)
    corrected_chunks = []
    for chunk in tqdm(chunks, desc="Correcting transcript"):
        corrected_chunks.append(correct_chunk(chunk))
    return " ".join(corrected_chunks)

# Example usage
if __name__ == "__main__":
    # Load your transcript from a file
    with open("transcript.txt", "r", encoding="utf-8") as f:
        transcript = f.read()

    corrected_transcript = correct_transcript(transcript)

    # Save corrected transcript
    with open("transcript_corrected.txt", "w", encoding="utf-8") as f:
        f.write(corrected_transcript)

    print("Transcript correction complete. Saved to transcript_corrected.txt")


/home/steven-dindl/Documents/AspireAI_LLM_Project/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-3.5-mini-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-3.5-mini-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does 